<a href="https://colab.research.google.com/github/Timixojo/Timixojo/blob/main/Electricity%20usage%20analysis_7005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Core libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ML libraries

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

In [ ]:
# Load Excel file

file_path = "/content/rural_data_with_summary.xlsx"

geo_rural = pd.read_excel(file_path, sheet_name="Geo_rural")
hh_rural = pd.read_excel(file_path, sheet_name="HH_rural")
apps_rural = pd.read_excel(file_path, sheet_name="HHapps_rural")

# Display Data Sheets
print("Geo Rural")
display(geo_rural)

#print("Household Rural")
#display(hh_rural)

#print("Appliance Rural")
display(apps_rural)

In [ ]:
# Data Cleaning Script

# Geo Rural Cleaning

# Drop rows with missing essential geographic information
geo_rural = geo_rural.dropna(subset=["zone", "state", "eaid", "urca_cat"])

# Keep only rows categorized as rural based on proximity to cities/towns
rural_values = ["<1hr to large city", "<1hr to int. city", "<1hr to small city/town+"]
geo_rural = geo_rural[geo_rural["urca_cat"].isin(rural_values)]

# Remove entries with non-positive household size (assuming hhsize should be at least 1)
geo_rural = geo_rural[geo_rural["hhsize"] > 0]

# Display Cleaned Geo Rural
print("Cleaned Geo Rural")
display(geo_rural)

# Household Rural Cleaning

# Keep entries with positive decoded household size (remove zero/negative)
hh_rural = hh_rural[hh_rural["hhsize_decoded"] > 0]

# Drop rows with missing decoded household income
hh_rural = hh_rural.dropna(subset=["hhincome_decoded"])

# Remove duplicate household entries based on hhid
hh_rural = hh_rural.drop_duplicates(subset=["hhid"])

# Display Cleaned Household Rural
print("Cleaned Household Rural")
display(hh_rural)

# Household Appliance Rural Cleaning

# Calculate total usage hours by summing q403_2 and q403_3, filling missing values with 0
apps_rural["usage_hours"] = apps_rural[["q403_2","q403_3"]].fillna(0).sum(axis=1)

# Keep only appliance entries with usage hours greater than 0
apps_rural = apps_rural[apps_rural["usage_hours"] > 0]

# Keep only appliance entries with realistic usage hours (<= 24)
apps_rural = apps_rural[apps_rural["usage_hours"] <= 24]

# Drop rows with missing household IDs in the appliance data
apps_rural = apps_rural.dropna(subset=["hhid"])

# Remove duplicate appliance entries (if any appliance instance is listed more than once for a household)
apps_rural = apps_rural.drop_duplicates()

# Display Cleaned Household Appliance Rural
print("Cleaned Household Appliance Rural")
display(apps_rural)

# Check unique appliance counts per household after cleaning
appliance_counts_after_cleaning = apps_rural.groupby('hhid')['q403'].nunique()

In [ ]:
# Save Cleaned Data( Geo Rural, Household Rural and Household Appliance Rural) Files

output_file = "/content/rural_data_cleaned_revised.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    geo_rural.to_excel(writer, sheet_name="Geo_rural_cleaned_revised", index=False)
    hh_rural.to_excel(writer, sheet_name="HH_rural_cleaned_revised", index=False)
    apps_rural.to_excel(writer, sheet_name="HHapps_rural_cleaned_revised", index=False)

print(f"\nCleaned data saved to: {output_file}")

In [ ]:
#Stat

print("Unique households (hhid):", hh_rural["hhid"].nunique())
print("Unique LGAs:", hh_rural["lga"].nunique())
print("Unique States:", hh_rural["state"].nunique())
print("Unique Zones:", hh_rural["zone"].nunique())

In [ ]:
# Exploratory Data Analysis

def show(title, df):
    print(f"\n{title}")
    display(df)

# GEO_RURAL_CLEANED (EDA)

# EAIDs per Zone
eaid_zone = geo_rural["zone"].value_counts().reset_index()
eaid_zone.columns = ["zone", "eaid_count"]

# EAIDs per State
eaid_state = geo_rural["state"].value_counts().reset_index()
eaid_state.columns = ["state", "eaid_count"]

# EAIDs per Rural Category
eaid_urcat = geo_rural["urca_cat"].value_counts().reset_index()
eaid_urcat.columns = ["urca_cat", "eaid_count"]

# EAID stats (count/unique/top/freq)
eaid_stats = geo_rural["eaid"].describe().to_frame("eaid_stats")

# Display
print("\neaid_zone:")
display(eaid_zone)

print("\neaid_state:")
display(eaid_state)

print("\neaid_urcat:")
display(eaid_urcat)

print("\neaid_stats:")
display(eaid_stats)

# HH_RURAL_CLEANED (EDA)

income_top = (hh_rural["hhincome_decoded"]
              .value_counts()
              .reset_index()
              .rename(columns={"index":"income_source","hhincome_decoded":"households"}))

hhsize_stats = hh_rural["hhsize_decoded"].describe().to_frame("hhsize_stats")

show("Income Categories (All, sorted)", income_top)
show("Household Size Summary (HH)", hhsize_stats)

# HHAPPS_RURAL_CLEANED (EDA)

# Ensure an 'appliance' alias and usage hours column exist (idempotent)
if "appliance" not in apps_rural.columns:
    apps_rural["appliance"] = apps_rural["q403"].astype(str)

if "usage_hours" not in apps_rural.columns:
    apps_rural["usage_hours"] = apps_rural[["q403_2","q403_3"]].fillna(0).sum(axis=1)

appliance_top = (apps_rural["appliance"]
                 .value_counts()
                 .reset_index()
                 .rename(columns={"index":"appliance","appliance":"count"}))

usage_by_appliance = (apps_rural
                      .groupby("appliance", as_index=False)["usage_hours"]
                      .mean()
                      .sort_values("usage_hours", ascending=False))

usage_per_hh = (apps_rural
                .groupby("hhid", as_index=False)["usage_hours"]
                .sum()
                .rename(columns={"usage_hours":"total_usage_hours"}))

usage_per_hh_stats = usage_per_hh["total_usage_hours"].describe().to_frame("total_usage_hours_stats")

show("Top Appliances (by records)", appliance_top.head(10))
show("Average Usage Hours per Appliance", usage_by_appliance.head(10))
show("Total Usage Hours per Household (summary)", usage_per_hh_stats)

In [ ]:
# EDA VISUALISATION

# GEO_RURAL: EAID distributions

# EAIDs per State
plt.figure(figsize=(8,4))
eaid_state.set_index("state")["eaid_count"].plot(kind="bar")
plt.title("EAIDs per State")
plt.ylabel("Number of EAIDs")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# EAIDs per Zone
plt.figure(figsize=(8,4))
eaid_zone.set_index("zone")["eaid_count"].plot(kind="bar")
plt.title("EAIDs per Zone")
plt.ylabel("Number of EAIDs")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# EAIDs per Rural Category
plt.figure(figsize=(6,6))
eaid_urcat.set_index("urca_cat")["eaid_count"].plot(
    kind="pie",
    autopct='%1.1f%%',
    startangle=90,
    legend=False
)
plt.title("EAIDs per Rural Category")
plt.ylabel("")  # remove y-label
plt.tight_layout()
plt.show()

# HOUSEHOLDS: Size + Income

# Household size distribution
plt.figure(figsize=(8,4))
hh_rural["hhsize_decoded"].plot(kind="hist", bins=15)
plt.title("Household Size Distribution")
plt.xlabel("Household size")
plt.ylabel("HH Count")
plt.tight_layout()
plt.show()

# Top income sources
income_counts = hh_rural["hhincome_decoded"].value_counts().head(10)
plt.figure(figsize=(8,4))
income_counts.sort_values(ascending=True).plot(kind="barh")
plt.title("Top Income Sources")
plt.xlabel("Number of Households")
plt.tight_layout()
plt.show()

# APPLIANCES: Ownership + Usage

# Top Appliances
appliance_counts = apps_rural["q403"].astype(str).value_counts().head(10)
plt.figure(figsize=(8,4))
appliance_counts.plot(kind="bar")
plt.title("Top Appliances")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Average usage hours per appliance
usage_by_app = (apps_rural
                .assign(appliance=apps_rural["q403"].astype(str))
                .groupby("appliance")["usage_hours"].mean()
                .sort_values(ascending=False)
                .head(10))
plt.figure(figsize=(8,4))
usage_by_app.sort_values(ascending=True).plot(kind="barh")
plt.title("Average Usage Hours per Appliance")
plt.xlabel("Average Hours per Day")
plt.tight_layout()
plt.show()

# Load vs Peak (household-level)

# Total device-hours per household (sum of all appliances)
total_device_hours = (apps_rural.groupby("hhid")["usage_hours"]
                      .sum()
                      .rename("total_device_hours"))

# Peak single-appliance hours per household (max; ≤ 24)
peak_single_appliance_hours = (apps_rural.groupby("hhid")["usage_hours"]
                               .max()
                               .rename("peak_single_appliance_hours"))

# Plot: total device-hours (hist; can exceed 24)
plt.figure(figsize=(8,4))
total_device_hours.plot(kind="hist", bins=20)
plt.title("Total Device-Hours per Household (Sum across appliances)")
plt.xlabel("Device-Hours per Day")
plt.ylabel("Households")
plt.tight_layout()
plt.show()

# Plot: peak single-appliance hours (hist; ≤ 24)
plt.figure(figsize=(8,4))
peak_single_appliance_hours.plot(kind="hist", bins=20)
plt.title("Peak Single-Appliance Usage per Household (≤ 24)")
plt.xlabel("Hours per Day (max per appliance)")
plt.ylabel("Households")
plt.tight_layout()
plt.show()

In [ ]:
# Features Engineering Process

# 1) Aggregate appliance data into household-level features
apps = apps_rural.copy()

# (a) Total usage hours per household (sum of all appliances)
hh_usage = apps.groupby("hhid")["usage_hours"].sum().reset_index()
hh_usage.rename(columns={"usage_hours": "total_usage_hours"}, inplace=True)

# (b) Number of unique appliance types per household
hh_unique_apps_count = apps.groupby("hhid")["q403"].nunique().reset_index()
hh_unique_apps_count.rename(columns={"q403": "unique_appliance_count"}, inplace=True)

# (c) Total number of appliance instances per household
hh_total_apps_count = apps.groupby("hhid").size().reset_index(name="total_appliance_instances")


# (d) Ownership flags for all appliance types
# Create dummy variables for all appliance types
ownership_flags = pd.get_dummies(apps[['hhid', 'q403']], columns=['q403'], prefix='owns')

# Aggregate the ownership flags by household, taking the maximum value (1 if owned, 0 if not)
ownership_flags = ownership_flags.groupby('hhid').max().reset_index()


# 2) Use HH_rural for household data (includes geo info, hhsize, and income)
# Include hhsize_decoded and hhincome_decoded as requested
hh_geo = hh_rural[["hhid", "lga", "state", "eaid", "zone", "hhsize_decoded", "hhincome_decoded"]].copy()


# 3) Merge all features into one household table
household_features = hh_geo.merge(hh_usage, on="hhid", how="left")
household_features = household_features.merge(hh_unique_apps_count, on="hhid", how="left")
household_features = household_features.merge(hh_total_apps_count, on="hhid", how="left") # Merge the new feature
# Merge the corrected ownership flags
household_features = household_features.merge(ownership_flags, on="hhid", how="left")

# Note: hhsize_decoded and hhincome_decoded are kept in the dataframe

# Remove rows with missing values in the features that WILL BE used for clustering
# This ensures that X_scaled does not contain NaNs
# The clustering features do NOT include hhsize_decoded or hhincome_decoded based on previous decisions
clustering_feature_cols = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances"] + [col for col in household_features.columns if col.startswith("owns_")]
household_features.dropna(subset=clustering_feature_cols, inplace=True)
print(f"\nShape of household_features after removing rows with missing values for clustering features: {household_features.shape}")


# 4) Normalize/Standardize features for clustering
# Use the defined clustering_feature_cols for scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(household_features[clustering_feature_cols])

# Display the final feature table and scaled data
display(household_features)

In [ ]:
# Using Elbow method and Dendrogram for Kmeans (K) and Agglomerative (n)

# Based on features from Feature Engineering (excluding hhsize_decoded in clustering_feature_cols)

def find_k(X, method="kmeans", k_range=range(2,10), linkage_method="ward"):
    if method == "kmeans":
        inertia_vals = []
        for k in k_range:
            model = KMeans(n_clusters=k, random_state=42, n_init=10)
            model.fit(X)
            inertia_vals.append(model.inertia_)

        # Plot Elbow Curve For KMeans
        plt.figure(figsize=(8,4))
        plt.plot(k_range, inertia_vals, marker='o')
        plt.title("Elbow")
        plt.xlabel("Number of clusters (k)")
        plt.ylabel("Inertia")
        plt.show()

    elif method == "agglomerative":
        # Plot Dendrogram for Agglomerative
        plt.figure(figsize=(8,4))
        Z = linkage(X, method=linkage_method)
        dendrogram(Z, truncate_mode="lastp", p=20, show_leaf_counts=True)
        plt.title(f"Dendrogram")
        plt.xlabel("Samples")
        plt.ylabel("Distance")
        plt.show()

# For KMeans (Elbow)
find_k(X_scaled, method="kmeans", k_range=range(2,10))

# For Agglomerative (Dendrogram)
find_k(X_scaled, method="agglomerative", linkage_method="ward")

In [ ]:
# Applying Clustering (KMeans, DBSCAN, and Agglomerative)

# Use the clustering_feature_cols list defined in the Feature Engineering cell
# This list now includes 'total_usage_hours', 'unique_appliance_count', 'total_appliance_instances', and ownership flags (excluding hhsize_decoded)


# Apply KMeans clustering (k=3)
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
household_features['kmeans_cluster'] = kmeans.fit_predict(X_scaled) # Fit and predict on scaled data

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
household_features['dbscan_cluster'] = dbscan.fit_predict(X_scaled) # Fit and predict on scaled data

# Apply Agglomerative Clustering (n=3)
agglo = AgglomerativeClustering(n_clusters=3)
household_features['agglo_cluster'] = agglo.fit_predict(X_scaled) # Fit and predict on scaled data

# Display final dataframe with clustering results (showing key features and cluster assignments)
display(household_features[['hhid'] + clustering_feature_cols + ['kmeans_cluster', 'dbscan_cluster', 'agglo_cluster']])

In [ ]:
# Plot Clusters
# Note: Plotting high-dimensional clusters on 2D scatter plots can be difficult
# We can attempt to plot using the first two features or the first two PCA components later.
# For a basic visualization, let's use total_usage_hours and unique_appliance_count if they are in clustering_feature_cols

plot_features = []
if 'total_usage_hours' in clustering_feature_cols and 'unique_appliance_count' in clustering_feature_cols:
    plot_features = ['total_usage_hours', 'unique_appliance_count']
elif 'total_usage_hours' in clustering_feature_cols and 'total_appliance_instances' in clustering_feature_cols:
     plot_features = ['total_usage_hours', 'total_appliance_instances']

if len(plot_features) == 2:
    # Plot KMeans clusters
    plt.figure(figsize=(6, 4))
    colors = ['red', 'green', 'blue', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive', 'cyan'] # Extended colors
    for cluster_id in sorted(household_features['kmeans_cluster'].unique()):
        if cluster_id != -1: # Exclude noise for plot legend if present in other methods
            color_index = cluster_id % len(colors) # Cycle through colors
            color = colors[color_index]
            plt.scatter(household_features[household_features['kmeans_cluster'] == cluster_id][plot_features[0]],
                        household_features[household_features['kmeans_cluster'] == cluster_id][plot_features[1]],
                        c=color,
                        label=f'Cluster {cluster_id}',
                        alpha=0.6) # Add some transparency
    plt.title(f'KMeans Clustering')
    plt.xlabel(plot_features[0])
    plt.ylabel(plot_features[1])
    plt.legend()
    plt.grid(False) # Remove grid
    plt.show()

    # Plot DBSCAN clusters
    plt.figure(figsize=(6, 4))
    for cluster_id in sorted(household_features['dbscan_cluster'].unique()):
        if cluster_id != -1:
            color_index = cluster_id % len(colors) # Cycle through colors
            color = colors[color_index]
            label = f'Cluster {cluster_id}'
            alpha = 0.6
        else:
            color = 'gray' # Color for noise points
            label = 'Noise'
            alpha = 0.4 # More transparency for noise

        plt.scatter(household_features[household_features['dbscan_cluster'] == cluster_id][plot_features[0]],
                    household_features[household_features['dbscan_cluster'] == cluster_id][plot_features[1]],
                    c=color,
                    label=label,
                    alpha=alpha)
    plt.title(f'DBSCAN')
    plt.xlabel(plot_features[0])
    plt.ylabel(plot_features[1])
    # plt.legend() # Removed legend for DBSCAN
    plt.grid(False) # Remove grid
    plt.show()

    # Plot Agglomerative Clustering clusters
    plt.figure(figsize=(6, 4))
    for cluster_id in sorted(household_features['agglo_cluster'].unique()):
         if cluster_id != -1: # Exclude noise for plot legend if present in other methods
            color_index = cluster_id % len(colors) # Cycle through colors
            color = colors[color_index]
            plt.scatter(household_features[household_features['agglo_cluster'] == cluster_id][plot_features[0]],
                        household_features[household_features['agglo_cluster'] == cluster_id][plot_features[1]],
                        c=color,
                        label=f'Cluster {cluster_id}',
                        alpha=0.6) # Add some transparency
    plt.title(f'Agglomerative Clustering')
    plt.xlabel(plot_features[0])
    plt.ylabel(plot_features[1])
    plt.legend()
    plt.grid(False) # Remove grid
    plt.show()
else:
    print("\nCannot plot clusters on a 2D scatter plot with the selected features.")
    print("Please ensure clustering_feature_cols contains at least two features suitable for plotting, or use PCA results for visualization.")

In [ ]:
# Evaluate KMeans clustering using Silhouette Score and Davies-Bouldin Index
kmeans_silhouette = silhouette_score(X_scaled, household_features['kmeans_cluster'])
kmeans_davies_bouldin = davies_bouldin_score(X_scaled, household_features['kmeans_cluster'])

# Output KMeans evaluation results
print(f"KMeans")
print(f"  Silhouette Score: {kmeans_silhouette:.4f}")
print(f"  Davies-Bouldin Index: {kmeans_davies_bouldin:.4f}")

# Evaluate Agglomerative Clustering using Silhouette Score and Davies-Bouldin Index
agglo_silhouette = silhouette_score(X_scaled, household_features['agglo_cluster'])
agglo_davies_bouldin = davies_bouldin_score(X_scaled, household_features['agglo_cluster'])

# Output Agglomerative Clustering evaluation results
print(f"\nAgglomerative")
print(f"  Silhouette Score: {agglo_silhouette:.4f}")
print(f"  Davies-Bouldin Index: {agglo_davies_bouldin:.4f}")

# Note: Evaluating DBSCAN requires different approaches due to the presence of noise.
# Metrics like Silhouette Score can be calculated on the non-noise points,
# but interpreting them requires careful consideration of how DBSCAN handles outliers.

In [ ]:
# Apply PCA

# Use the clustering_feature_cols list defined in the Feature Engineering cell

# Apply PCA to reduce dimensions (e.g., to 2 components)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(household_features[clustering_feature_cols])  # Apply PCA on the correct feature columns

# Clustering with PCA
# Apply KMeans clustering (k=3) on the PCA-reduced data
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)
kmeans_labels = kmeans.fit_predict(X_pca)

# Apply DBSCAN clustering on the PCA-reduced data
# You may need to tune eps and min_samples based on your data
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_pca)

# Apply Agglomerative Clustering (n=3) on the PCA-reduced data
agglo = AgglomerativeClustering(n_clusters=3)
agglo_labels = agglo.fit_predict(X_pca)

# Assign clustering results to the household_features dataframe
# Ensure that household_features still has the correct index for merging
household_features['kmeans_pca_cluster'] = kmeans_labels
household_features['dbscan_pca_cluster'] = dbscan_labels # Add DBSCAN cluster labels
household_features['agglo_pca_cluster'] = agglo_labels

# Display Clustering with PCA
print("\nHousehold Features with PCA Cluster")
display(household_features[['hhid'] + clustering_feature_cols + ['kmeans_pca_cluster', 'dbscan_pca_cluster', 'agglo_pca_cluster']])

In [ ]:
# Plot Clusters with PCA

# Plot the clusters for KMeans
plt.figure(figsize=(6, 4))
# Use the cluster labels for coloring
scatter_kmeans = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels, cmap='viridis', alpha=0.6)
plt.title('KMeans')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(*scatter_kmeans.legend_elements(), title="Clusters")
plt.grid(False) # Remove grid
plt.show()

# Plot the clusters for DBSCAN
plt.figure(figsize=(6, 4))
# Use the cluster labels for coloring
scatter_dbscan = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=dbscan_labels, cmap='viridis', alpha=0.6)
plt.title('DBSCAN')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
#plt.legend(*scatter_dbscan.legend_elements(), title="Clusters") # Removed legend
plt.grid(False) # Remove grid
plt.show()


# Plot the clusters for Agglomerative
plt.figure(figsize=(6, 4))
# Use the cluster labels for coloring
scatter_agglo = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=agglo_labels, cmap='viridis', alpha=0.6)
plt.title('Agglomerative Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(*scatter_agglo.legend_elements(), title="Clusters")
plt.grid(False) # Remove grid
plt.show()

In [ ]:
# Evaluation metrics

# Evaluate KMeans clustering using Silhouette Score and Davies-Bouldin Index
silhouette_kmeans = silhouette_score(X_pca, kmeans_labels)
dbi_kmeans = davies_bouldin_score(X_pca, kmeans_labels)

# Evaluate DBSCAN clustering using Silhouette Score and Davies-Bouldin Index
# Note: Silhouette Score for DBSCAN is typically calculated on non-noise points
# and interpretation needs care. Davies-Bouldin can be calculated on all points.
# We'll calculate on all points for simplicity here, but be mindful of interpretation.
if len(set(dbscan_labels)) > 1: # Need more than 1 cluster to calculate silhouette
    silhouette_dbscan = silhouette_score(X_pca, dbscan_labels)
    dbi_dbscan = davies_bouldin_score(X_pca, dbscan_labels)
else:
    silhouette_dbscan = None # Cannot calculate silhouette with only one cluster or all noise
    dbi_dbscan = None


# Evaluate Agglomerative Clustering using Silhouette Score and Davies-Bouldin Index
silhouette_agglo = silhouette_score(X_pca, agglo_labels)
dbi_agglo = davies_bouldin_score(X_pca, agglo_labels)

# Print evaluation results for clustering with PCA
print("Clustering with PCA")

print(f"\nKMeans")
print(f"  Silhouette Score: {silhouette_kmeans:.4f}")
print(f"  Davies-Bouldin Index: {dbi_kmeans:.4f}")

print(f"\nDBSCAN")
if silhouette_dbscan is not None:
    print(f"  Silhouette Score: {silhouette_dbscan:.4f}")
else:
    print("  Silhouette Score: Cannot be calculated (only one cluster or all noise)")

if dbi_dbscan is not None:
    print(f"  Davies-Bouldin Index: {dbi_dbscan:.4f}")
else:
     print("  Davies-Bouldin Index: Cannot be calculated (only one cluster or all noise)")

print(f"\nAgglomerative")
print(f"  Silhouette Score: {silhouette_agglo:.4f}")
print(f"  Davies-Bouldin Index: {dbi_agglo:.4f}")

In [ ]:
# Analyze the characteristics of KMeans PCA clusters

# Define the features to include in the cluster analysis
analysis_features = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances"] + [col for col in household_features.columns if col.startswith("owns_")]

# Group by KMeans PCA cluster and calculate the mean of the analysis features
kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[analysis_features].mean()

print("KMeans PCA Cluster Analysis")
display(kmeans_pca_cluster_analysis)

# For binary ownership flags (owns_*), the mean represents the proportion of households in the cluster that own the appliance.

In [ ]:
# @title unique_appliance_count vs total_appliance_instances

from matplotlib import pyplot as plt
kmeans_pca_cluster_analysis.plot(kind='scatter', x='unique_appliance_count', y='total_appliance_instances', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title total_usage_hours vs unique_appliance_count

from matplotlib import pyplot as plt
kmeans_pca_cluster_analysis.plot(kind='scatter', x='total_usage_hours', y='unique_appliance_count', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title total_usage_hours

from matplotlib import pyplot as plt
kmeans_pca_cluster_analysis['total_usage_hours'].plot(kind='hist', bins=20, title='total_usage_hours')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Visualize KMeans PCA Cluster Characteristics

# Plotting selected features for comparison
features_to_plot = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances", "owns_Fan", "owns_Television", "owns_Laptop / Computer", "owns_Light bulb"]

kmeans_pca_cluster_analysis[features_to_plot].T.plot(kind='bar', figsize=(6, 6))
plt.title('Mean Feature Values per KMeans PCA Cluster')
plt.xlabel('Feature')
plt.ylabel('Mean Value')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

In [ ]:
# You can create individual plots for more detailed comparison if needed

# For example, plotting total_usage_hours per cluster:
kmeans_pca_cluster_analysis['total_usage_hours'].plot(kind='bar', figsize=(6,4))
plt.title('Average Total Usage Hours per KMeans PCA Cluster')
plt.xlabel('Cluster')
plt.ylabel('Average Total Usage Hours')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Geographic Analysis of KMeans PCA Clusters

print("Geographic Distribution of KMeans PCA Clusters")

# Cross-tabulation of Cluster vs. Zone
print("\nCluster distribution across Zones:")
cluster_zone_A = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'])
display(cluster_zone_A)

print("\nCluster proportion across Zones:")
cluster_zone_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_prop)

# Cross-tabulation of Cluster vs. State
print("\nCluster distribution across States:")
cluster_state_A = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'])
display(cluster_state_A)

print("\nCluster proportion across States:")
cluster_state_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'], normalize='index') # Proportions within each cluster summing to 1
display(cluster_state_prop)

# Analyze distribution by LGA for a deeper look
print("\nCluster distribution across LGAs:")
cluster_lga = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['lga'])
display(cluster_lga)

# Analyzing by EAID is also possible but will result in a very large table
# For EAID:
# cluster_eaid = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['eaid'])
# display(cluster_eaid)

In [ ]:
# Deep Dive: Analyze hhsize_decoded distribution per KMeans PCA Cluster

print("Household Size (hhsize_decoded) Analysis per KMeans PCA Cluster")

# Group by cluster and describe the hhsize_decoded distribution
hhsize_analysis = household_features.groupby('kmeans_pca_cluster')['hhsize_decoded'].describe()

print("\nHousehold Size Distribution Summary per Cluster:")
display(hhsize_analysis)

# Optional: Plotting the distribution (e.g., box plots) for better visualization
plt.figure(figsize=(8, 5))
sns.boxplot(x='kmeans_pca_cluster', y='hhsize_decoded', data=household_features)
plt.title('Household Size Distribution per KMeans PCA Cluster')
plt.xlabel('KMeans PCA Cluster')
plt.ylabel('Household Size (hhsize_decoded)')
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
# Deep Dive: Analyze hhincome_decoded distribution per KMeans PCA Cluster

print("Household Income (hhincome_decoded) Analysis per KMeans PCA Cluster")

# Diagnosis: Print columns before crosstab
print("\nColumns in household_features before crosstab:")
print(household_features.columns)
# End Diagnosis


# Cross-tabulation of Cluster vs. Household Income (counts)
print("\nCluster distribution across Household Income Categories (Counts):")
cluster_income_counts = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'])
display(cluster_income_counts)

# Cross-tabulation of Cluster vs. Household Income (proportions within each cluster)
print("\nCluster proportion across Household Income Categories (Proportions within each cluster):")
cluster_income_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='index') # Proportions within each cluster summing to 1
display(cluster_income_prop)

# Optional: You could also normalize='columns' to see the proportion of each income category within each cluster.
# print("\nProportion of each Income Category within each Cluster:")
# cluster_income_prop_cols = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='columns')
# display(cluster_income_prop_cols)

In [ ]:
# Synthesize information and describe each KMeans PCA cluster

print("Detailed Profile of KMeans PCA Clusters:\n")

# Recreate analysis variables
# Define the features to include in the cluster analysis
analysis_features = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances"] + [col for col in household_features.columns if col.startswith("owns_")]

# Group by KMeans PCA cluster and calculate the mean of the analysis features
kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[analysis_features].mean()

# Geographic Analysis variables
cluster_zone_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'], normalize='index')
cluster_state_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'], normalize='index')
cluster_lga = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['lga'])

# Household Size Analysis variable
hhsize_analysis = household_features.groupby('kmeans_pca_cluster')['hhsize_decoded'].describe()

# Household Income Analysis variables
cluster_income_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='index')


# Get the unique cluster IDs
cluster_ids = sorted(household_features['kmeans_pca_cluster'].unique())

# Loop through each cluster to generate the profile
for cluster_id in cluster_ids:
    print(f"--- Cluster {cluster_id} Profile ---")

    # 1. Appliance Ownership and Usage
    print("\nAppliance Ownership and Usage:")
    cluster_features = kmeans_pca_cluster_analysis.loc[cluster_id]

    print(f"  - Average Total Usage Hours: {cluster_features['total_usage_hours']:.2f}")
    print(f"  - Average Unique Appliance Count: {cluster_features['unique_appliance_count']:.2f}")
    print(f"  - Average Total Appliance Instances: {cluster_features['total_appliance_instances']:.2f}")

    print("  - Appliance Ownership Proportions:")
    ownership_cols = [col for col in kmeans_pca_cluster_analysis.columns if col.startswith("owns_")]
    for col in ownership_cols:
        appliance_name = col.replace("owns_", "")
        print(f"    - {appliance_name}: {cluster_features[col]:.2f}")

    # 2. Geographic Location
    print("\nGeographic Location:")
    cluster_zone_dist = cluster_zone_prop.loc[cluster_id].sort_values(ascending=False)
    cluster_state_dist = cluster_state_prop.loc[cluster_id].sort_values(ascending=False).head() # Show top 5 states
    cluster_lga_dist = cluster_lga.loc[cluster_id].sort_values(ascending=False).head() # Show top 5 LGAs by count


    print("  - Distribution by Zone (Proportion):")
    display(cluster_zone_dist.to_frame().T)

    print("  - Top 5 States (Proportion within cluster):")
    display(cluster_state_dist.to_frame().T)

    print("  - Top 5 LGAs (Count):")
    display(cluster_lga_dist.to_frame().T)


    # 3. Household Size
    print("\nHousehold Size (hhsize_decoded):")
    hhsize_desc = hhsize_analysis.loc[cluster_id]
    print(f"  - Mean: {hhsize_desc['mean']:.2f}")
    print(f"  - Median: {hhsize_desc['50%']:.2f}")
    print(f"  - Std Dev: {hhsize_desc['std']:.2f}")
    print(f"  - Min: {hhsize_desc['min']:.2f}")
    print(f"  - Max: {hhsize_desc['max']:.2f}")


    # 4. Household Income
    print("\nHousehold Income (hhincome_decoded):")
    income_dist = cluster_income_prop.loc[cluster_id].sort_values(ascending=False)
    print("  - Distribution by Income Source (Proportion within cluster):")
    display(income_dist.to_frame().T)

    print("\n" + "="*50 + "\n") # Separator for clarity

In [ ]:
# Geographic Analysis for KMeans (with PCA)
print("\nGeographic Distribution of KMeans PCA Clusters")
print("\nCluster proportion across Zones (KMeans PCA):")
cluster_zone_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_kmeans_pca)

print("\nCluster proportion across States (KMeans PCA):")
cluster_state_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'], normalize='index')
display(cluster_state_kmeans_pca)

print("\nCluster distribution across LGAs (KMeans PCA - Counts):")
cluster_lga_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['lga'])
display(cluster_lga_kmeans_pca)


# Geographic Analysis for Agglomerative (with PCA)
print("\nGeographic Distribution of Agglomerative PCA Clusters")
print("\nCluster proportion across Zones (Agglomerative PCA):")
cluster_zone_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_agglo_pca)

print("\nCluster proportion across States (Agglomerative PCA):")
cluster_state_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['state'], normalize='index')
display(cluster_state_agglo_pca)

print("\nCluster distribution across LGAs (Agglomerative PCA - Counts):")
cluster_lga_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['lga'])
display(cluster_lga_agglo_pca)


# Geographic Analysis for DBSCAN (with PCA)
print("\nGeographic Distribution of DBSCAN PCA Clusters")
# Exclude noise points (-1) for meaningful geographic analysis of core clusters
dbscan_pca_core_clusters_geo = household_features[household_features['dbscan_pca_cluster'] != -1]

if not dbscan_pca_core_clusters_geo.empty:
    print("\nCluster proportion across Zones (DBSCAN PCA - Core Clusters):")
    cluster_zone_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['zone'], normalize='index')
    display(cluster_zone_dbscan_pca)

    print("\nCluster proportion across States (DBSCAN PCA - Core Clusters):")
    cluster_state_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['state'], normalize='index')
    display(cluster_state_dbscan_pca)

    print("\nCluster distribution across LGAs (DBSCAN PCA - Core Clusters - Counts):")
    cluster_lga_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['lga'])
    display(cluster_lga_dbscan_pca)
else:
    print("No core clusters found in DBSCAN (with PCA) results for geographic analysis.")


# Household Size Analysis
print("\nHousehold Size (hhsize_decoded) Analysis per Cluster")

# KMeans
print("\nHousehold Size Distribution Summary per KMeans Cluster:")
hhsize_kmeans = household_features.groupby('kmeans_cluster')['hhsize_decoded'].describe()
display(hhsize_kmeans)

# Agglomerative
print("\nHousehold Size Distribution Summary per Agglomerative Cluster:")
hhsize_agglo = household_features.groupby('agglo_cluster')['hhsize_decoded'].describe()
display(hhsize_agglo)

# DBSCAN (Excluding noise)
print("\nHousehold Size Distribution Summary per DBSCAN Cluster (Excluding Noise):")
if not dbscan_core_clusters.empty:
    hhsize_dbscan = dbscan_core_clusters.groupby('dbscan_cluster')['hhsize_decoded'].describe()
    display(hhsize_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for household size analysis.")

# KMeans PCA
print("\nHousehold Size Distribution Summary per KMeans PCA Cluster:")
hhsize_kmeans_pca = household_features.groupby('kmeans_pca_cluster')['hhsize_decoded'].describe()
display(hhsize_kmeans_pca)

# Agglomerative PCA
print("\nHousehold Size Distribution Summary per Agglomerative PCA Cluster:")
hhsize_agglo_pca = household_features.groupby('agglo_pca_cluster')['hhsize_decoded'].describe()
display(hhsize_agglo_pca)

# DBSCAN PCA (Excluding noise)
print("\nHousehold Size Distribution Summary per DBSCAN PCA Cluster (Excluding Noise):")
if not dbscan_pca_core_clusters.empty:
    hhsize_dbscan_pca = dbscan_pca_core_clusters.groupby('dbscan_pca_cluster')['hhsize_decoded'].describe()
    display(hhsize_dbscan_pca)
else:
     print("No core clusters found in DBSCAN (with PCA) results for household size analysis.")


# Household Income Analysis
print("\nHousehold Income (hhincome_decoded) Analysis per Cluster")

# KMeans
print("\nCluster proportion across Household Income Categories (KMeans):")
income_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_kmeans)

# Agglomerative
print("\nCluster proportion across Household Income Categories (Agglomerative):")
income_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_agglo)

# DBSCAN (Excluding noise)
print("\nCluster proportion across Household Income Categories (DBSCAN - Core Clusters):")
if not dbscan_core_clusters.empty:
    income_dbscan = pd.crosstab(dbscan_core_clusters['dbscan_cluster'], dbscan_core_clusters['hhincome_decoded'], normalize='index')
    display(income_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for income analysis.")


# KMeans PCA
print("\nCluster proportion across Household Income Categories (KMeans PCA):")
income_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_kmeans_pca)

# Agglomerative PCA
print("\nCluster proportion across Household Income Categories (Agglomerative PCA):")
income_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_agglo_pca)

# DBSCAN PCA (Excluding noise)
print("\nCluster proportion across Household Income Categories (DBSCAN PCA - Core Clusters):")
if not dbscan_pca_core_clusters.empty:
    income_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters['dbscan_pca_cluster'], dbscan_pca_core_clusters['hhincome_decoded'], normalize='index')
    display(income_dbscan_pca)
else:
    print("No core clusters found in DBSCAN (with PCA) results for income analysis.")

In [ ]:
# Geographic Analysis for KMeans (without PCA)
print("\nGeographic Distribution of KMeans Clusters (without PCA)")
print("\nCluster proportion across Zones (KMeans):")
cluster_zone_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_kmeans)

print("\nCluster proportion across States (KMeans):")
cluster_state_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['state'], normalize='index')
display(cluster_state_kmeans)

print("\nCluster distribution across LGAs (KMeans - Counts):")
# Displaying counts for LGA as proportions can be too granular
cluster_lga_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['lga'])
display(cluster_lga_kmeans)


# Geographic Analysis for Agglomerative (without PCA)
print("\nGeographic Distribution of Agglomerative Clusters (without PCA)")
print("\nCluster proportion across Zones (Agglomerative):")
cluster_zone_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_agglo)

print("\nCluster proportion across States (Agglomerative):")
cluster_state_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['state'], normalize='index')
display(cluster_state_agglo)

print("\nCluster distribution across LGAs (Agglomerative - Counts):")
cluster_lga_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['lga'])
display(cluster_lga_agglo)


# Geographic Analysis for DBSCAN (without PCA)
print("\nGeographic Distribution of DBSCAN Clusters (without PCA)")
# Exclude noise points (-1) for meaningful geographic analysis of core clusters
dbscan_core_clusters_geo = household_features[household_features['dbscan_cluster'] != -1]

if not dbscan_core_clusters_geo.empty:
    print("\nCluster proportion across Zones (DBSCAN - Core Clusters):")
    cluster_zone_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['zone'], normalize='index')
    display(cluster_zone_dbscan)

    print("\nCluster proportion across States (DBSCAN - Core Clusters):")
    cluster_state_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['state'], normalize='index')
    display(cluster_state_dbscan)

    print("\nCluster distribution across LGAs (DBSCAN - Core Clusters - Counts):")
    cluster_lga_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['lga'])
    display(cluster_lga_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for geographic analysis.")

In [ ]:
# 8) Calculate mean feature values for KMeans clusters (with PCA)
print("\nMean Feature Values per KMeans Cluster (with PCA)")
kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[clustering_feature_cols].mean()
display(kmeans_pca_cluster_analysis)

# 9) Calculate mean feature values for Agglomerative clusters (with PCA)
print("\nMean Feature Values per Agglomerative Cluster (with PCA)")
agglo_pca_cluster_analysis = household_features.groupby('agglo_pca_cluster')[clustering_feature_cols].mean()
display(agglo_pca_cluster_analysis)

# 10) Calculate mean feature values for DBSCAN clusters (with PCA)
# Exclude noise points (-1) for mean calculation
print("\nMean Feature Values per DBSCAN Cluster (with PCA, excluding noise)")
dbscan_pca_core_clusters = household_features[household_features['dbscan_pca_cluster'] != -1]
if not dbscan_pca_core_clusters.empty:
    dbscan_pca_cluster_analysis = dbscan_pca_core_clusters.groupby('dbscan_pca_cluster')[clustering_feature_cols].mean()
    display(dbscan_pca_cluster_analysis)
else:
    print("No core clusters found in DBSCAN (with PCA) results to calculate mean features.")

In [ ]:
# 5) Calculate mean feature values for KMeans clusters (without PCA)
print("\nMean Feature Values per KMeans Cluster (without PCA)")
kmeans_cluster_analysis = household_features.groupby('kmeans_cluster')[clustering_feature_cols].mean()
display(kmeans_cluster_analysis)

# 6) Calculate mean feature values for Agglomerative clusters (without PCA)
print("\nMean Feature Values per Agglomerative Cluster (without PCA)")
agglo_cluster_analysis = household_features.groupby('agglo_cluster')[clustering_feature_cols].mean()
display(agglo_cluster_analysis)

# 7) Calculate mean feature values for DBSCAN clusters (without PCA)
# Exclude noise points (-1) for mean calculation
print("\nMean Feature Values per DBSCAN Cluster (without PCA, excluding noise)")
dbscan_core_clusters = household_features[household_features['dbscan_cluster'] != -1]
if not dbscan_core_clusters.empty:
    dbscan_cluster_analysis = dbscan_core_clusters.groupby('dbscan_cluster')[clustering_feature_cols].mean()
    display(dbscan_cluster_analysis)
else:
    print("No core clusters found in DBSCAN results (without PCA) to calculate mean features.")

In [ ]:
# 1) Cross-tabulation of KMeans vs. Agglomerative Clustering (without PCA)
print("Cross-tabulation: KMeans Clusters vs. Agglomerative Clusters (without PCA)")
crosstab_kmeans_agglo = pd.crosstab(household_features['kmeans_cluster'], household_features['agglo_cluster'])
display(crosstab_kmeans_agglo)

# 2) Cross-tabulation of KMeans vs. DBSCAN (without PCA)
print("\nCross-tabulation: KMeans Clusters vs. DBSCAN Clusters (without PCA)")
crosstab_kmeans_dbscan = pd.crosstab(household_features['kmeans_cluster'], household_features['dbscan_cluster'])
display(crosstab_kmeans_dbscan)

# 3) Cross-tabulation of Agglomerative Clustering vs. DBSCAN (without PCA)
print("\nCross-tabulation: Agglomerative Clusters vs. DBSCAN Clusters (without PCA)")
crosstab_agglo_dbscan = pd.crosstab(household_features['agglo_cluster'], household_features['dbscan_cluster'])
display(crosstab_agglo_dbscan)

In [ ]:
# 5) Calculate mean feature values for KMeans clusters (without PCA)
print("\nMean Feature Values per KMeans Cluster (without PCA)")
kmeans_cluster_analysis = household_features.groupby('kmeans_cluster')[clustering_feature_cols].mean()
display(kmeans_cluster_analysis)

# 6) Calculate mean feature values for Agglomerative clusters (without PCA)
print("\nMean Feature Values per Agglomerative Cluster (without PCA)")
agglo_cluster_analysis = household_features.groupby('agglo_cluster')[clustering_feature_cols].mean()
display(agglo_cluster_analysis)

# 7) Calculate mean feature values for DBSCAN clusters (without PCA)
# Exclude noise points (-1) for mean calculation
print("\nMean Feature Values per DBSCAN Cluster (without PCA, excluding noise)")
dbscan_core_clusters = household_features[household_features['dbscan_cluster'] != -1]
if not dbscan_core_clusters.empty:
    dbscan_cluster_analysis = dbscan_core_clusters.groupby('dbscan_cluster')[clustering_feature_cols].mean()
    display(dbscan_cluster_analysis)
else:
    print("No core clusters found in DBSCAN results (without PCA) to calculate mean features.")

In [ ]:
# 8) Calculate mean feature values for KMeans clusters (with PCA)
print("\nMean Feature Values per KMeans Cluster (with PCA)")
kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[clustering_feature_cols].mean()
display(kmeans_pca_cluster_analysis)

# 9) Calculate mean feature values for Agglomerative clusters (with PCA)
print("\nMean Feature Values per Agglomerative Cluster (with PCA)")
agglo_pca_cluster_analysis = household_features.groupby('agglo_pca_cluster')[clustering_feature_cols].mean()
display(agglo_pca_cluster_analysis)

# 10) Calculate mean feature values for DBSCAN clusters (with PCA)
# Exclude noise points (-1) for mean calculation
print("\nMean Feature Values per DBSCAN Cluster (with PCA, excluding noise)")
dbscan_pca_core_clusters = household_features[household_features['dbscan_pca_cluster'] != -1]
if not dbscan_pca_core_clusters.empty:
    dbscan_pca_cluster_analysis = dbscan_pca_core_clusters.groupby('dbscan_pca_cluster')[clustering_feature_cols].mean()
    display(dbscan_pca_cluster_analysis)
else:
    print("No core clusters found in DBSCAN (with PCA) results to calculate mean features.")

In [ ]:
# Geographic Analysis for KMeans (without PCA)
print("\nGeographic Distribution of KMeans Clusters (without PCA)")
print("\nCluster proportion across Zones (KMeans):")
cluster_zone_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_kmeans)

print("\nCluster proportion across States (KMeans):")
cluster_state_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['state'], normalize='index')
display(cluster_state_kmeans)

print("\nCluster distribution across LGAs (KMeans - Counts):")
# Displaying counts for LGA as proportions can be too granular
cluster_lga_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['lga'])
display(cluster_lga_kmeans)


# Geographic Analysis for Agglomerative (without PCA)
print("\nGeographic Distribution of Agglomerative Clusters (without PCA)")
print("\nCluster proportion across Zones (Agglomerative):")
cluster_zone_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_agglo)

print("\nCluster proportion across States (Agglomerative):")
cluster_state_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['state'], normalize='index')
display(cluster_state_agglo)

print("\nCluster distribution across LGAs (Agglomerative - Counts):")
cluster_lga_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['lga'])
display(cluster_lga_agglo)


# Geographic Analysis for DBSCAN (without PCA)
print("\nGeographic Distribution of DBSCAN Clusters (without PCA)")
# Exclude noise points (-1) for meaningful geographic analysis of core clusters
dbscan_core_clusters_geo = household_features[household_features['dbscan_cluster'] != -1]

if not dbscan_core_clusters_geo.empty:
    print("\nCluster proportion across Zones (DBSCAN - Core Clusters):")
    cluster_zone_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['zone'], normalize='index')
    display(cluster_zone_dbscan)

    print("\nCluster proportion across States (DBSCAN - Core Clusters):")
    cluster_state_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['state'], normalize='index')
    display(cluster_state_dbscan)

    print("\nCluster distribution across LGAs (DBSCAN - Core Clusters - Counts):")
    cluster_lga_dbscan = pd.crosstab(dbscan_core_clusters_geo['dbscan_cluster'], dbscan_core_clusters_geo['lga'])
    display(cluster_lga_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for geographic analysis.")

In [ ]:
# Geographic Analysis for KMeans (with PCA)
print("\nGeographic Distribution of KMeans PCA Clusters")
print("\nCluster proportion across Zones (KMeans PCA):")
cluster_zone_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_kmeans_pca)

print("\nCluster proportion across States (KMeans PCA):")
cluster_state_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'], normalize='index')
display(cluster_state_kmeans_pca)

print("\nCluster distribution across LGAs (KMeans PCA - Counts):")
cluster_lga_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['lga'])
display(cluster_lga_kmeans_pca)


# Geographic Analysis for Agglomerative (with PCA)
print("\nGeographic Distribution of Agglomerative PCA Clusters")
print("\nCluster proportion across Zones (Agglomerative PCA):")
cluster_zone_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['zone'], normalize='index')
display(cluster_zone_agglo_pca)

print("\nCluster proportion across States (Agglomerative PCA):")
cluster_state_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['state'], normalize='index')
display(cluster_state_agglo_pca)

print("\nCluster distribution across LGAs (Agglomerative PCA - Counts):")
cluster_lga_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['lga'])
display(cluster_lga_agglo_pca)


# Geographic Analysis for DBSCAN (with PCA)
print("\nGeographic Distribution of DBSCAN PCA Clusters")
# Exclude noise points (-1) for meaningful geographic analysis of core clusters
dbscan_pca_core_clusters_geo = household_features[household_features['dbscan_pca_cluster'] != -1]

if not dbscan_pca_core_clusters_geo.empty:
    print("\nCluster proportion across Zones (DBSCAN PCA - Core Clusters):")
    cluster_zone_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['zone'], normalize='index')
    display(cluster_zone_dbscan_pca)

    print("\nCluster proportion across States (DBSCAN PCA - Core Clusters):")
    cluster_state_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['state'], normalize='index')
    display(cluster_state_dbscan_pca)

    print("\nCluster distribution across LGAs (DBSCAN PCA - Core Clusters - Counts):")
    cluster_lga_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters_geo['dbscan_pca_cluster'], dbscan_pca_core_clusters_geo['lga'])
    display(cluster_lga_dbscan_pca)
else:
    print("No core clusters found in DBSCAN (with PCA) results for geographic analysis.")


# Household Size Analysis
print("\nHousehold Size (hhsize_decoded) Analysis per Cluster")

# KMeans
print("\nHousehold Size Distribution Summary per KMeans Cluster:")
hhsize_kmeans = household_features.groupby('kmeans_cluster')['hhsize_decoded'].describe()
display(hhsize_kmeans)

# Agglomerative
print("\nHousehold Size Distribution Summary per Agglomerative Cluster:")
hhsize_agglo = household_features.groupby('agglo_cluster')['hhsize_decoded'].describe()
display(hhsize_agglo)

# DBSCAN (Excluding noise)
print("\nHousehold Size Distribution Summary per DBSCAN Cluster (Excluding Noise):")
if not dbscan_core_clusters.empty:
    hhsize_dbscan = dbscan_core_clusters.groupby('dbscan_cluster')['hhsize_decoded'].describe()
    display(hhsize_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for household size analysis.")

# KMeans PCA
print("\nHousehold Size Distribution Summary per KMeans PCA Cluster:")
hhsize_kmeans_pca = household_features.groupby('kmeans_pca_cluster')['hhsize_decoded'].describe()
display(hhsize_kmeans_pca)

# Agglomerative PCA
print("\nHousehold Size Distribution Summary per Agglomerative PCA Cluster:")
hhsize_agglo_pca = household_features.groupby('agglo_pca_cluster')['hhsize_decoded'].describe()
display(hhsize_agglo_pca)

# DBSCAN PCA (Excluding noise)
print("\nHousehold Size Distribution Summary per DBSCAN PCA Cluster (Excluding Noise):")
if not dbscan_pca_core_clusters.empty:
    hhsize_dbscan_pca = dbscan_pca_core_clusters.groupby('dbscan_pca_cluster')['hhsize_decoded'].describe()
    display(hhsize_dbscan_pca)
else:
     print("No core clusters found in DBSCAN (with PCA) results for household size analysis.")


# Household Income Analysis
print("\nHousehold Income (hhincome_decoded) Analysis per Cluster")

# KMeans
print("\nCluster proportion across Household Income Categories (KMeans):")
income_kmeans = pd.crosstab(household_features['kmeans_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_kmeans)

# Agglomerative
print("\nCluster proportion across Household Income Categories (Agglomerative):")
income_agglo = pd.crosstab(household_features['agglo_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_agglo)

# DBSCAN (Excluding noise)
print("\nCluster proportion across Household Income Categories (DBSCAN - Core Clusters):")
if not dbscan_core_clusters.empty:
    income_dbscan = pd.crosstab(dbscan_core_clusters['dbscan_cluster'], dbscan_core_clusters['hhincome_decoded'], normalize='index')
    display(income_dbscan)
else:
    print("No core clusters found in DBSCAN results (without PCA) for income analysis.")


# KMeans PCA
print("\nCluster proportion across Household Income Categories (KMeans PCA):")
income_kmeans_pca = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_kmeans_pca)

# Agglomerative PCA
print("\nCluster proportion across Household Income Categories (Agglomerative PCA):")
income_agglo_pca = pd.crosstab(household_features['agglo_pca_cluster'], household_features['hhincome_decoded'], normalize='index')
display(income_agglo_pca)

# DBSCAN PCA (Excluding noise)
print("\nCluster proportion across Household Income Categories (DBSCAN PCA - Core Clusters):")
if not dbscan_pca_core_clusters.empty:
    income_dbscan_pca = pd.crosstab(dbscan_pca_core_clusters['dbscan_pca_cluster'], dbscan_pca_core_clusters['hhincome_decoded'], normalize='index')
    display(income_dbscan_pca)
else:
    print("No core clusters found in DBSCAN (with PCA) results for income analysis.")

In [ ]:
# Display the mean feature values per KMeans PCA cluster
# This table was generated during the analysis phase (cell id: f4c3a34f, also recreated in LMBgg9j_XcF7)
# Recreate the analysis_features list if it's not available in this session
if 'analysis_features' not in locals():
    analysis_features = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances"] + [col for col in household_features.columns if col.startswith("owns_")]

kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[analysis_features].mean()
display(kmeans_pca_cluster_analysis)

In [ ]:
# Display the mean feature values per KMeans PCA cluster
# This table was generated during the analysis phase (cell id: f4c3a34f, also recreated in LMBgg9j_XcF7)
# Recreate the analysis_features list if it's not available in this session
if 'analysis_features' not in locals():
    analysis_features = ["total_usage_hours", "unique_appliance_count", "total_appliance_instances"] + [col for col in household_features.columns if col.startswith("owns_")]

kmeans_pca_cluster_analysis = household_features.groupby('kmeans_pca_cluster')[analysis_features].mean()
display(kmeans_pca_cluster_analysis)

In [ ]:
# Display the geographic distribution of KMeans PCA clusters
# These tables were generated during the analysis phase (cell id: jR7seXqx-Zff)
# Recreate geographic analysis variables if not available
if 'cluster_zone_prop' not in locals() or 'cluster_state_prop' not in locals() or 'cluster_lga' not in locals():
    cluster_zone_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['zone'], normalize='index')
    cluster_state_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['state'], normalize='index')
    cluster_lga = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['lga'])

print("Cluster proportion across Zones:")
display(cluster_zone_prop)

print("\nCluster proportion across States:")
display(cluster_state_prop)

print("\nCluster distribution across LGAs (Counts):")
display(cluster_lga)

In [ ]:
# Display the household size distribution summary per KMeans PCA cluster
# This table and plot were generated during the analysis phase (cell id: 13FefYGJ-40r)
# Recreate hhsize_analysis variable if not available
if 'hhsize_analysis' not in locals():
    hhsize_analysis = household_features.groupby('kmeans_pca_cluster')['hhsize_decoded'].describe()

print("\nHousehold Size Distribution Summary per Cluster:")
display(hhsize_analysis)

# Replot the box plot for household size distribution
plt.figure(figsize=(8, 5))
sns.boxplot(x='kmeans_pca_cluster', y='hhsize_decoded', data=household_features)
plt.title('Household Size Distribution per KMeans PCA Cluster')
plt.xlabel('KMeans PCA Cluster')
plt.ylabel('Household Size (hhsize_decoded)')
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
# Display the household income distribution per KMeans PCA cluster
# These tables were generated during the analysis phase (cell id: c143b78f)
# Recreate cluster_income_prop variable if not available
if 'cluster_income_prop' not in locals():
    cluster_income_prop = pd.crosstab(household_features['kmeans_pca_cluster'], household_features['hhincome_decoded'], normalize='index')

print("\nCluster proportion across Household Income Categories (Proportions within each cluster):")
display(cluster_income_prop)